# Proyecto Final: Web Scraping de E‑commerce (Oxylabs Sandbox)

Este cuaderno implementa **todo el flujo** solicitado: listado ➜ detalle ➜ combinación ➜ DataFrame/CSV ➜ *opcional* carga a **Azure Blob Storage**.

## Cómo usar
1. Ejecuta la celda de **Instalación** (en Colab ya viene `pandas`/`requests`, pero incluimos por si acaso).
2. Ejecuta la celda de **Configuración y utilidades**.
3. Ejecuta **Paso 1: Función de listado** y **Paso 2: Función de detalle**.
4. Ejecuta **Paso 3: Orquestación y combinación** (recolecta >= 5 páginas por defecto).
5. Ejecuta **Paso 4: Guardar a DataFrame y CSV**.
6. (Opcional) Ejecuta **Carga a Azure Blob Storage** (configura la variable de entorno `AZURE_STORAGE_CONNECTION_STRING` o pega el *connection string* en la función de carga).

> **Nota**: Este sitio es un _sandbox_ público para prácticas de scraping (Oxylabs) y puede variar sutilmente su HTML con el tiempo. El código incluye validaciones y valores por defecto.

---
**Objetivo**
- Extraer de https://sandbox.oxylabs.io/products la información de cada producto:
  - *Título*, *Géneros*, *Descripción*, *Precio*, *Desarrollador*, *Tipo*, *URL*.
- Navegar por **≥ 5 páginas** del catálogo.
- Exportar resultados a **`productos.csv`**.
- Subir el CSV a un contenedor de **Azure Blob Storage** (p. ej. `scraping-data`).

---
## Estructura del cuaderno
- **1. Preparar el notebook para la extracción**
  - a. Explorar y cargar la página en el notebook.
  - b. Entender la estructura HTML objetivo.
- **2. Estrategia de extracción**
  - Paso 1: Función de **listado**
  - Paso 2: Función de **detalle**
  - Paso 3: **Combinar** información
  - Paso 4: **DataFrame** y **CSV**
- **3. Cargar CSV a Azure Blob Storage (opcional)**
- **4. Verificar resultados en Azure** (realizado desde el Portal)


In [ ]:
# === Instalación (si estás en Colab, esto normalmente funciona tal cual) ===
import sys
IN_COLAB = 'google.colab' in sys.modules

def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0].split('[')[0])
    except Exception:
        !pip -q install {pkg}

for p in [
    'requests',
    'beautifulsoup4',
    'lxml',
    'pandas',
    'tqdm',
    'azure-storage-blob'
]:
    pip_install(p)

print('✔️ Instalación verificada')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.9/428.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.3/213.3 kB 13.9 MB/s eta 0:00:00
✔️ Instalación verificada


In [ ]:
# === Configuración y utilidades ===
import os
import math
import time
import random
from typing import List, Dict
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

BASE_URL = 'https://sandbox.oxylabs.io'
CATALOG_URL = f'{BASE_URL}/products'

# Sesión HTTP con encabezados y reintentos sencillos
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8'
})

def get_soup(url: str, max_retries: int = 3, backoff: float = 1.0) -> BeautifulSoup:
    last_exc = None
    for attempt in range(1, max_retries + 1):
        try:
            resp = session.get(url, timeout=20)
            resp.raise_for_status()
            return BeautifulSoup(resp.text, 'lxml')
        except Exception as e:
            last_exc = e
            time.sleep(backoff * attempt)
    raise last_exc

def abs_url(href: str) -> str:
    if not href:
        return ''
    if href.startswith('http://') or href.startswith('https://'):
        return href
    return BASE_URL.rstrip('/') + '/' + href.lstrip('/')

print('✔️ Configuración lista')

✔️ Configuración lista


## 1. Preparar el notebook para la extracción
### a) Explorar y cargar la página en el notebook
A continuación validamos la **petición HTTP** y que podemos **parsear el HTML**.

In [ ]:
soup_test = get_soup(CATALOG_URL)
print('Título de la página:', soup_test.title.get_text(strip=True) if soup_test.title else 'N/A')
print('Primeros 200 caracteres del HTML:\n', soup_test.get_text(strip=True)[:200], '...')

Título de la página: E-commerce	| Oxylabs Scraping Sandbox
Primeros 200 caracteres del HTML:
 E-commerce	| Oxylabs Scraping SandboxGame platforms:AllNintendo platformwiiwii-unintendo-64switchgamecubegame-boy-advance3dsdsXbox platformDreamcastPlaystation platformPcStadiaNote!This is a sandbox w ...


### b) Entendiendo la estructura de la página
Cada producto está contenido en un bloque similar a:
```html
<div class="product-card">
  <a class="card-header" href="/products/1"> ... </a>
  <h4 class="title">Título</h4>
  <p class="category">
    <span>Acción</span><span>Aventura</span>
  </p>
</div>
```
Y el **detalle** contiene `Developer`, `Type`, `Description`, `Price`, etc.

## 2. Estrategia de extracción
### Paso 1: Función de listado (recorre páginas del catálogo y extrae básicos)

In [ ]:
def scrape_listing_page(url: str) -> List[Dict]:
    """Devuelve una lista de dicts con: titulo, url, generos (string) y descripcion resumen si está disponible."""
    soup = get_soup(url)
    cards = soup.find_all('div', class_='product-card')
    results = []
    for card in cards:
        # Título
        title_el = card.find('h4', class_='title')
        titulo = title_el.get_text(strip=True) if title_el else 'N/A'
        # URL detalle
        a_el = card.find('a', class_='card-header')
        href = a_el.get('href') if a_el else ''
        url_abs = abs_url(href)
        # Géneros
        cat_p = card.find('p', class_='category')
        generos = []
        if cat_p:
            for span in cat_p.select('span'):
                txt = span.get_text(strip=True)
                if txt:
                    generos.append(txt)
        generos_str = ', '.join(generos) if generos else 'N/A'
        # Descripción breve si existiera en la tarjeta
        desc_el = card.find('p', class_='description')
        desc_short = desc_el.get_text(strip=True) if desc_el else ''

        results.append({
            'Titulo': titulo,
            'URL': url_abs,
            'Generos': generos_str,
            'Descripcion_listado': desc_short
        })
    return results

def scrape_listings_n_pages(base_catalog_url: str, n_pages: int = 5, page_param: str = 'page') -> List[Dict]:
    """Recorre n páginas de catálogo. Asume paginación con query-string ?page=2,3,..."""
    all_rows = []
    for page in tqdm(range(1, n_pages + 1), desc='Listados'):
        url = base_catalog_url if page == 1 else f"{base_catalog_url}?{page_param}={page}"
        page_rows = scrape_listing_page(url)
        all_rows.extend(page_rows)
        time.sleep(random.uniform(0.5, 1.2))  # pequeña espera entre páginas
    return all_rows

print('✔️ Funciones de listado listas')

✔️ Funciones de listado listas


### Paso 2: Función de detalle (visita cada producto y extrae campos faltantes)
Se recoge: **Developer**, **Type**, **Descripción** (ampliada), **Precio**.

In [ ]:
def clean_prefix(text: str, prefix: str) -> str:
    if not text:
        return ''
    t = text.strip()
    if t.lower().startswith(prefix.lower()):
        return t[len(prefix):].strip(': ').strip()
    return t

def scrape_detail(url: str) -> Dict:
    soup = get_soup(url)
    # Developer y Type suelen estar en brand-wrapper
    dev = 'N/A'
    typ = 'N/A'
    brand_wrapper = soup.find('div', class_='brand-wrapper')
    if brand_wrapper:
        spans = brand_wrapper.find_all('span')
        for sp in spans:
            txt = sp.get_text(strip=True)
            if not txt:
                continue
            low = txt.lower()
            if 'developer' in low:
                dev = clean_prefix(txt, 'Developer')
            elif 'type' in low:
                typ = clean_prefix(txt, 'Type')

    # Descripción
    desc_el = soup.find('p', class_='description')
    desc = desc_el.get_text(strip=True) if desc_el else 'N/A'

    # Precio
    price_el = soup.find('div', class_='price')
    price = price_el.get_text(strip=True) if price_el else 'N/A'

    return {
        'Developer': dev if dev else 'N/A',
        'Type': typ if typ else 'N/A',
        'Descripcion': desc if desc else 'N/A',
        'Precio': price if price else 'N/A'
    }

def enrich_with_details(rows: List[Dict], pause_range=(0.4, 1.1)) -> List[Dict]:
    enriched = []
    for row in tqdm(rows, desc='Detalles'):
        url = row.get('URL', '')
        detail = {}
        try:
            if url:
                detail = scrape_detail(url)
        except Exception as e:
            detail = {'Developer': 'N/A', 'Type': 'N/A', 'Descripcion': 'N/A', 'Precio': 'N/A'}
        merged = {**row, **detail}
        enriched.append(merged)
        time.sleep(random.uniform(*pause_range))  # pequeña espera entre solicitudes
    return enriched

print('✔️ Funciones de detalle listas')

✔️ Funciones de detalle listas


### Paso 3: Combinar la información (flujo principal)
Orquestamos los pasos anteriores para construir la **lista completa de productos**.

In [ ]:
def run_scraper(n_pages: int = 5) -> List[Dict]:
    listings = scrape_listings_n_pages(CATALOG_URL, n_pages=n_pages)
    print(f'Total productos listados: {len(listings)}')
    data = enrich_with_details(listings)
    print(f'Total productos con detalle: {len(data)}')
    return data

print('✔️ Orquestador listo')

✔️ Orquestador listo


### Paso 4: Guardar en un DataFrame y exportar a CSV

In [ ]:
def to_dataframe(rows: List[Dict]) -> pd.DataFrame:
    df = pd.DataFrame(rows)
    # Reordenamos columnas a un orden lógico
    preferred = ['Titulo','Generos','Developer','Type','Precio','Descripcion','Descripcion_listado','URL']
    cols = [c for c in preferred if c in df.columns] + [c for c in df.columns if c not in preferred]
    df = df[cols]
    return df

def save_csv(df: pd.DataFrame, path: str = 'productos.csv') -> str:
    df.to_csv(path, index=False, encoding='utf-8')
    return os.path.abspath(path)

# Ejemplo de corrida completa (descomenta para ejecutar real):
data = run_scraper(n_pages=5)
df = to_dataframe(data)
csv_path = save_csv(df, 'productos.csv')
df.head()

Listados: 100%|██████████| 5/5 [00:07<00:00,  1.56s/it]


Total productos listados: 160


Detalles: 100%|██████████| 160/160 [02:20<00:00,  1.14it/s]

Total productos con detalle: 160


,Titulo,Generos,Developer,Type,Precio,Descripcion,Descripcion_listado,URL
0,The Legend of Zelda: Ocarina of Time,"Action Adventure, Fantasy",Nintendo,singleplayer,"91,99 €","As a young boy, Link is tricked by Ganondorf, ...","As a young boy, Link is tricked by Ganondorf, ...",https://sandbox.oxylabs.io/products/1
1,Super Mario Galaxy,"Action, Platformer, 3D",Nintendo,singleplayer,"91,99 €",[Metacritic's 2007 Wii Game of the Year] The u...,[Metacritic's 2007 Wii Game of the Year] The u...,https://sandbox.oxylabs.io/products/2
2,Super Mario Galaxy 2,"Action, Platformer, 3D",Nintendo EAD Tokyo,singleplayer,"91,99 €","Super Mario Galaxy 2, the sequel to the galaxy...","Super Mario Galaxy 2, the sequel to the galaxy...",https://sandbox.oxylabs.io/products/3
3,Metroid Prime,"Action, Shooter, First-Person, Sci-Fi",Retro Studios,singleplayer,"89,99 €",Samus returns in a new mission to unravel the ...,Samus returns in a new mission to unravel the ...,https://sandbox.oxylabs.io/products/4
4,Super Mario Odyssey,"Action, Platformer, 3D",Nintendo,singleplayer,"89,99 €",New Evolution of Mario Sandbox-Style Gameplay....,New Evolution of Mario Sandbox-Style Gameplay....,https://sandbox.oxylabs.io/products/5


In [ ]:
from google.colab import files
files.download("productos.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. Cargando el archivo en Azure Blob Storage (opcional)
Configura la variable de entorno `AZURE_STORAGE_CONNECTION_STRING` con el *connection string* de tu **Storage Account**, o pásalo como argumento a la función.

In [37]:
import os
from azure.storage.blob import BlobServiceClient

def upload_to_azure(blob_name='productos.csv', container_name='scraping-data', connection_string=None):
    # 1) Obtener el connection string (arg > env)
    if connection_string is None:
        connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING', '').strip()

    # 2) Validación clara del formato
    required = ('DefaultEndpointsProtocol=', 'AccountName=', 'AccountKey=', 'EndpointSuffix=')
    if not connection_string or not all(p in connection_string for p in required):
        env_val = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
        if env_val is None or not env_val.strip():
            raise ValueError(
                "AZURE_STORAGE_CONNECTION_STRING no está configurado o el argumento 'connection_string' está vacío.\n"
                "Copia el *Connection string* completo desde Azure Portal → Storage account → Access keys."
            )
        else:
            raise ValueError(
                "El valor proporcionado NO es un *connection string* válido. Debe incluir:\n"
                "DefaultEndpointsProtocol, AccountName, AccountKey y EndpointSuffix (una sola línea)."
            )

    # 3) Paranoia anti-placeholders / caracteres inválidos
    bad_tokens = ("<", ">", "%3e", "@", "\n", "\r")
    if any(t in connection_string for t in bad_tokens):
        raise ValueError(
            "Connection string contiene placeholders o caracteres inválidos (<, >, %3e, @ o saltos de línea).\n"
            "Vuelve a copiarlo tal cual del Portal (Access keys → Connection string)."
        )

    # 4) Conexión y nombre de cuenta (diagnóstico útil)
    bsc = BlobServiceClient.from_connection_string(connection_string)
    print("Usando cuenta:", bsc.account_name)

    # 5) Contenedor (minúsculas, sin espacios)
    container_name = container_name.strip().lower()
    cc = bsc.get_container_client(container_name)
    try:
        cc.create_container()
        print(f"Contenedor '{container_name}' listo")
    except Exception:
        print(f"Contenedor '{container_name}' ya existe")

    # 6) Archivo local
    if not os.path.exists(blob_name):
        raise FileNotFoundError(f"No se encontró el archivo local: {blob_name}")

    # 7) Subida
    with open(blob_name, "rb") as f:
        cc.upload_blob(name=blob_name, data=f, overwrite=True)
    print(f"✔️ Subido {blob_name} a contenedor '{container_name}'")

os.environ["AZURE_STORAGE_CONNECTION_STRING"] = "DefaultEndpointsProtocol=https;AccountName=storagewebscrappingac;AccountKey=VPRTp1P3yeE0zZB9s281HAHQo9UDsifVPwuVTMiDJfUGxTi9VhNjsTGxuX+8+r7SyhtIh5hZ3d8K+AStcEUzBA==;EndpointSuffix=core.windows.net"
upload_to_azure('productos.csv', container_name='scraping-data')


Usando cuenta: storagewebscrappingac
Contenedor 'scraping-data' ya existe
✔️ Subido productos.csv a contenedor 'scraping-data'


## 4. Verificar los resultados en Azure (Portal)
1. Ve a **Azure Portal → Storage Account → Containers**.
2. Abre el contenedor **`scraping-data`**.
3. Verifica que se encuentre `productos.csv` con la **fecha/hora** de carga.

---
### Consejos de validación
- Revisa `df.head()` y `df.info()` tras la recolección.
- Comprueba que el número de filas coincida con la suma de productos de las páginas listadas.
- Inspecciona valores `N/A` para verificar qué campos faltan en ciertos productos.

### Ética de scraping (buenas prácticas)
- Usa *User-Agent* y esperas pequeñas entre solicitudes (implementado).
- Maneja errores y faltantes con gracia.
- Este *sandbox* es para prácticas educativas; para sitios reales, revisa **robots.txt** y Términos de Uso.


## 5. DATOS

1. Profesora
   ANGELA PATRICIA VILLOTA GOMEZ
2. Estudiante: Rubén Darío Sabogal urbano
4. Practica de WebScraping
5. Materia : Extraccion de datos.
6. Maestria de IA
7. Universidad ICESI
8. Octubre de 2025 - cali